In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import math
import numpy as np
import random
import tensorflow as tf
from tensorflow import keras

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

### Load data

In [4]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
#mnist = tf.keras.datasets.mnist
#(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
#cifar10 = tf.keras.datasets.cifar10
#(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
#train_labels = train_labels[:, 0]
#test_labels = test_labels[:, 0]

Normalization

In [ ]:
train_images = tf.cast(train_images/255, dtype=tf.float32)
test_images = tf.cast(test_images /255, dtype=tf.float32)

Data flattening

In [6]:
x_train = tf.reshape(train_images, [train_images.shape[0], -1])
x_test = tf.reshape(test_images, [test_images.shape[0], -1])

Onehot representation

In [7]:
y_train = tf.one_hot(train_labels, 10)
y_test = tf.one_hot(test_labels, 10)

Transformation of the data range as ([0,1]->[-1,1]) 

In [8]:
x_train = 2*x_train - 1
x_test = 2*x_test - 1
y_train = 2*y_train - 1
y_test = 2*y_test - 1

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)
plt.show()

### Quantum DRBM (QDRBM)

In [10]:
from layers import *
from qdrbm import DRBM
from loss import negative_loglikelihood

In [11]:
initializer = 'glorot_uniform'
quantum_initializer = 'glorot_uniform'

In [12]:
model = DRBM(num_classes=10, weight_hidden=10.0)
model.add(InputLayer(num_input_units=x_train[0].shape[0]))
#model.add(RBMLayer(500, 
#    weight_initializer=initializer))
model.add(QRBMLayer(500, 
    weight_initializer=initializer))
model.add(RBMLayer(10, weight_initializer=initializer))

In [ ]:
model.summary()

In [14]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    #run_eagerly=True,
    loss=[negative_loglikelihood],
    metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [15]:
def lr_scheduler(epoch):
    lr = 0.005 / (epoch+1)
    if lr < 1e-4:
        return 1e-4
    else: return lr

#lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)
#es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)
#callbacks = [es_callback]
#callbacks = [lr_callback]
callbacks = []

In [ ]:
history = model.fit(
    x_train,
    y_train,
    epochs=100, 
    validation_split=0.1,
    batch_size=100,
    callbacks=callbacks)

In [ ]:
results = model.evaluate(
    x_test,
    tf.one_hot(test_labels, 10))
print(results)

In [ ]:
import pandas as pd
pd.DataFrame(history.history)[['loss', 'val_loss']].plot()